In [1]:
###Python Packages###

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib.colors import LogNorm
import math as mth

import timeit
import ODE_utils as odeu
from scipy import optimize
import tables
from scipy.integrate import cumtrapz as trapint
#from mpmath import *
!jt -t onedork -cellw 100%

### Evolution Code

In [2]:
###Initial Values###

M0=1.0   ###inital mass
Q=0.95   ###charge
Lambda=0.0000   ###cosmological constant
scalarfield=False   ###whether scalar field is applied at u=0 boundary
A=.01   ###amplitude of scalar field
vscalarfield=False   ###legacy, will eventually control whether scalar field is applied along v=0


In [3]:
N=40  ###Resolution in v-direction
Tol=10**(-0)/N  ###Determines the error tolerance above which the code will reduce the step size
Nul=5*N   ### Base Resolution in u-direction
scal=50  ###extra scaling on the u=0 boundary to reduce error from boundary condtions there
uscal=20   ###how often results are printed/saved to file (saved every uscal points)
umax=100   ###max value of u that the code will go to
vmax=40   ###max value of v that the code will go to
extraterms=True ###saves the derivatives of r,sig,phi to hard drive if True
rcosmtol=.00000001 ###how close the code will let r get to rcosm if u=0 initial conditions include cosmological horizon

datatype=np.float64   ###legacy, previous versions used decimal 

ru0=2   ###initial value of r at (u=0,v=0)

dr0v=.1   ###the value of dr/dr at (u=0,v=0), not choosable for eddington coordinates

vbdytype="edd"   ###coordinate choice in v-direction (can choose "stan" standard, "edd" eddington)
ubdytype="stan"   ###coordinate choice in v-direction (can choose "stan" standard, "edd" eddington, "adapt" adaptive gauge) 

if vbdytype=="edd":
    dr0v=0.5*(1-2*M0/ru0+Q**2.0/ru0**2.0-Lambda/3*ru0**2.0)  ###forces dr0v to be a certain value for eddington coordinates
    
print(dr0v)


print("Max error is: "+str(Tol*umax))

0.1128125
Max error is: 2.5


In [4]:
###Defining Needed Constants###

scalf=float(scal)
u0=0.0
v0=0.0
rv0=ru0
phiu0=0.0
phiv0=0.0

if M0==0.0:
    dv0=1/N    
else:
    dv0=M0/N
    


def fr(r,M0,Q,Lambda):
    return 1-2*M0/r+Q**2.0/r**2.0-Lambda*r**2.0/3.0

if Lambda>0 or Lambda<0:
    sol = optimize.root(fr,[0.687,1.3,500.0],args=(M0,Q,Lambda), method='hybr')
    rminus=sol.x[0]
    rplus=sol.x[1]
    rcosm=sol.x[2]
else:
    rplus=M0+(M0**2-Q**2)**(.5)
    rminus=M0-(M0**2-Q**2)**(.5)
    rcosm=0.0
    

wHD=True 

print("r+: "+str(rplus))
print("r-: "+str(rminus))
print("rc: "+str(rcosm))

print("Max r along u=0: "+str(ru0+vmax*dr0v))
###Predicting Run time###

###timer
start = timeit.default_timer()

Nu=Nul*umax#int(umax/du0)
Nv=int(vmax/dv0)


r+: 1.31224989991992
r-: 0.6877501000800801
rc: 0.0
Max r along u=0: 6.5125


In [5]:
print((rplus-rminus)/(2*rminus)*rcosm/rminus)

0.0


In [ ]:
##################################
###Applying Boundary Conditions###

rnpf=np.zeros((2,Nv),dtype=datatype)
signpf=np.zeros((2,Nv),dtype=datatype)
phinpf=np.zeros((2,Nv),dtype=datatype)
drnpvf=np.zeros((2,Nv),dtype=datatype)
dsignpvf=np.zeros((2,Nv),dtype=datatype)
dphinpvf=np.zeros((2,Nv),dtype=datatype)
drnpuf=np.zeros((2,Nv),dtype=datatype)
dsignpuf=np.zeros((2,Nv),dtype=datatype)
dphinpuf=np.zeros((2,Nv),dtype=datatype)
massnpf=np.full((2,Nv),M0)#np.zeros((2,Nv))
#drunpv=np.full((1,Nv-1),-1/(4*dr0v)*(1-2*M0/ru0+Q**2/ru0**2))

rnpf[0], signpf[0], phinpf[0] , drnpvf[0], dsignpvf[0], dphinpvf[0], drnpuf[0], dsignpuf[0], dphinpuf[0] , massnpf[0]= odeu.boundaryv(scal,ubdytype,vbdytype,Nv,ru0,dr0v,dv0,vmax,M0,Q,Lambda,scalarfield,A,rcosmtol,datatype)


mask=~np.isnan(rnpf[0])
print(Nv)
rnpf=rnpf[:,mask]

Nv=int(len(rnpf[0]))
rnpf=rnpf.reshape(2,Nv)
signpf=signpf[:,mask]#.reshape(2,Nv)
phinpf=phinpf[:,mask]#.reshape(2,Nv)
dphinpuf=dphinpuf[:,mask]#.reshape(2,Nv)  
dphinpvf=dphinpvf[:,mask]#.reshape(2,Nv)
dsignpuf=dsignpuf[:,mask]#.reshape(2,Nv)
dsignpvf=dsignpvf[:,mask]#.reshape(2,Nv)
drnpuf=drnpuf[:,mask]#.reshape(2,Nv)
drnpvf=drnpvf[:,mask]#.reshape(2,Nv)
massnpf=massnpf[:,mask]#.reshape(2,Nv)
print(Nv)
#print(massnpf)

dr/dv is 0.1128125


In [ ]:
vmax=(Nv)*dv0

vrange=np.arange(0,vmax-.00000001*vmax,dv0)#*rnpf[0])/rnpf[0]

    
plt.plot(vrange,rnpf[0])
#plt.plot(dphinpvf[0])
#plt.plot(dphinpuf[0])
Mc=np.nanmax(massnpf[0])
rcosm=odeu.rc(Mc,Q,Lambda)
plt.plot([0,vmax],[rcosm,rcosm],linestyle='--',color='green')
plt.show()



#print(vrange)
print(Mc,rcosm)
print(drnpuf[0,0])

print(drnpvf[0,0])

In [ ]:
drunptemp=np.empty((Nv))*np.nan

###Applying Propagation Algorithm###

urange=np.array([0.0],dtype=datatype)



i=0
icount=0
dv=dv0
if M0>0:
    du0=M0/Nul
    #du0=M0/N
elif M0==0:
    du0=1/Nul
    #du0=1/N
du=du0

    
atom = tables.Float64Atom()

rnpfile = tables.open_file('rnp.h5', mode='w',encoding="utf8") 
rnp = rnpfile.create_earray(rnpfile.root, 'data', atom, (0, Nv))
signpfile = tables.open_file('signp.h5', mode='w',encoding="utf8") 
signp = signpfile.create_earray(signpfile.root, 'data', atom, (0, Nv))
phinpfile = tables.open_file('phinp.h5', mode='w',encoding="utf8") 
phinp = phinpfile.create_earray(phinpfile.root, 'data', atom, (0, Nv))
massnpfile = tables.open_file('massnp.h5', mode='w',encoding="utf8") 
massnp = massnpfile.create_earray(massnpfile.root, 'data', atom, (0, Nv))

if extraterms==True:
    drnpufile = tables.open_file('drnpu.h5', mode='w',encoding="utf8") 
    drnpu = drnpufile.create_earray(drnpufile.root, 'data', atom, (0, Nv))
    drnpvfile = tables.open_file('drnpv.h5', mode='w',encoding="utf8") 
    drnpv = drnpvfile.create_earray(drnpvfile.root, 'data', atom, (0, Nv))
    dsignpufile = tables.open_file('dsignpu.h5', mode='w',encoding="utf8") 
    dsignpu = dsignpufile.create_earray(dsignpufile.root, 'data', atom, (0, Nv))
    dsignpvfile = tables.open_file('dsignpv.h5', mode='w',encoding="utf8") 
    dsignpv = dsignpvfile.create_earray(dsignpvfile.root, 'data', atom, (0, Nv))
    dphinpufile = tables.open_file('dphinpu.h5', mode='w',encoding="utf8") 
    dphinpu = dphinpufile.create_earray(dphinpufile.root, 'data', atom, (0, Nv))
    dphinpvfile = tables.open_file('dphinpv.h5', mode='w',encoding="utf8") 
    dphinpv = dphinpvfile.create_earray(dphinpvfile.root, 'data', atom, (0, Nv))
    
rnp.append(rnpf[0].reshape(1,Nv))
signp.append(signpf[0].reshape(1,Nv))
phinp.append(phinpf[0].reshape(1,Nv))
massnp.append(massnpf[0].reshape(1,Nv))

if extraterms==True:
    drnpu.append(drnpuf[0].reshape(1,Nv))
    drnpv.append(drnpvf[0].reshape(1,Nv))
    dsignpu.append(dsignpuf[0].reshape(1,Nv))
    dsignpv.append(dsignpvf[0].reshape(1,Nv))
    dphinpu.append(dphinpuf[0].reshape(1,Nv))
    dphinpv.append(dphinpvf[0].reshape(1,Nv))
    
drunptemp=np.nanmax(np.abs(drnpuf[0]))     
dsigunpvalue=np.nanmax(np.abs(dsignpuf[0]))
dphiutemp=np.nanmax(np.abs(dphinpuf[0]))
druunptemp=np.nanmax(np.abs(odeu.Coneq(drnpuf[0],dsignpuf[0],dphinpuf[0],rnpf[0])))

i=0
TempTol=1 
TempTolv=1
Eres=False
if ubdytype=='adapt':
    maxsig1=signpf[0][0]-np.nanmax(signpf[0])
    maxsig2=signpf[0][0]-np.nanmax(signpf[0])
    maxsig3=signpf[0][0]-np.nanmax(signpf[0])
    print(maxsig1,maxsig2,maxsig3)
   
du=du0

while max(urange)<umax and i<2000000 and du>0.0: #and rnpf[1][0]>0.0 :


    du = np.min([du,1/(1*Nul)])
    
    ###############
    ###Predictor###
    ###############
    drnpvf[1] = drnpvf[0]+du*odeu.Rfunc(drnpvf[0],drnpuf[0],rnpf[0],signpf[0],Q,Lambda)
    dsignpvf[1] = dsignpvf[0]+du*odeu.Sigfunc(drnpvf[0],drnpuf[0],dphinpuf[0],dphinpvf[0],rnpf[0],signpf[0],Q)
    dphinpvf[1] = dphinpvf[0]+du*odeu.Phifunc(drnpvf[0],drnpuf[0],dphinpuf[0],dphinpvf[0],rnpf[0])
    drnpuf[1] = drnpuf[0]+du*odeu.Coneq(drnpuf[0],dsignpuf[0],dphinpuf[0],rnpf[0])
    
    rnpf[1] = rnpf[0]+1/2*du*(drnpuf[0]+drnpuf[1])
    
    if ubdytype=='adapt':
        #nextsig=(maxsig1*3.0-maxsig2*3.0+maxsig3*1.0)
        #dsignpuf[1][0]=-np.nanmax(np.abs(dsignpuf[0]))/Nul
        dsignpuf[1][0] = -(signpf[1][-1]-np.log(drnpvf[1][-1])+3/2*np.log(2))
    elif ubdytype=='edd':
        #dsignpuf[1][0]=(np.log(abs(1-2*M0/rnpf[1][0]+Q**2.0/rnpf[1][0]-Lambda*rnpf[1][0]**2.0/3))-signpf[0][0])/du
        dsignpuf[1][0] = 2*(3*Q**2-3*M0*rnpf[1][0]+rnpf[1][0]**4*Lambda)/(rnpf[1][0]*(-3*Q**2+rnpf[1][0]*(6*M0-3*rnpf[1][0]+rnpf[1][0]**3*Lambda)))*drnpuf[1][0]
    
    elif ubdytype=='stan':
        dsignpuf[1][0] = 0.0
    
    signpf[1] = signpf[0]+du*dsignpuf[0]
    phinpf[1] = phinpf[0]+du*dphinpuf[0]
    
    dphinpuf[1] = 1/(rnpf[1])*(rnpf[1][0]*dphinpuf[1][0]-dv*trapint(drnpuf[1]*dphinpvf[1],initial=0))
    Sigfuncnp = odeu.Sigfunc(drnpvf[1],drnpuf[1],dphinpuf[1],dphinpvf[1],rnpf[1],signpf[1],Q)
    dsignpuf[1] = dsignpuf[1][0]+dv*trapint(Sigfuncnp,initial=0)
 
    ###Coarse Grid (to estimate error)###
    drnpvf2 = drnpvf[0]+2*du*odeu.Rfunc(drnpvf[0],drnpuf[0],rnpf[0],signpf[0],Q,Lambda)
    dsignpvf2 = dsignpvf[0]+2*du*odeu.Sigfunc(drnpvf[0],drnpuf[0],dphinpuf[0],dphinpvf[0],rnpf[0],signpf[0],Q)
    dphinpvf2 = dphinpvf[0]+2*du*odeu.Phifunc(drnpvf[0],drnpuf[0],dphinpuf[0],dphinpvf[0],rnpf[0])
    drnpuf2 = drnpuf[0]+2*du*odeu.Coneq(drnpuf[0],dsignpuf[0],dphinpuf[0],rnpf[0])
    
    rnpf2 = rnpf[0]+du*(drnpuf[0]+drnpuf2)
    signpf2 = signpf[0]+2*du*dsignpuf[0]
    phinpf2 = phinpf[0]+2*du*dphinpuf[0]
    
    dphinpuf2 = 1/(rnpf2)*(rnpf2[0]*dphinpuf[1][0]-dv*trapint(drnpuf2*dphinpvf2,initial=0))
    Sigfuncnp = odeu.Sigfunc(drnpvf2,drnpuf2,dphinpuf2,dphinpvf2,rnpf2,signpf2,Q)
    dsignpuf2 = dsignpuf[1][0]+dv*trapint(Sigfuncnp,initial=0)
    ###
    
    drnpvf2 = drnpvf[0]+du*(odeu.Rfunc(drnpvf[0],drnpuf[0],rnpf[0],signpf[0],Q,Lambda)+odeu.Rfunc(drnpvf2,drnpuf2,rnpf2,signpf2,Q,Lambda))
    dsignpvf2 = dsignpvf[0]+du*(odeu.Sigfunc(drnpvf[0],drnpuf[0],dphinpuf[0],dphinpvf[0],rnpf[0],signpf[0],Q)+odeu.Sigfunc(drnpvf2,drnpuf2,dphinpuf2,dphinpvf2,rnpf2,signpf2,Q))
    dphinpvf2 = dphinpvf[0]+du*(odeu.Phifunc(drnpvf[0],drnpuf[0],dphinpuf[0],dphinpvf[0],rnpf[0])+odeu.Phifunc(drnpvf2,drnpuf2,dphinpuf2,dphinpvf2,rnpf2))
    drnpuf2 = drnpuf[0]+du*(odeu.Coneq(drnpuf[0],dsignpuf[0],dphinpuf[0],rnpf[0])+odeu.Coneq(drnpuf2,dsignpuf2,dphinpuf2,rnpf2))
    
    rnpf2 = rnpf[0]+du*(drnpuf[0]+drnpuf2)
    signpf2 = signpf[0]+du*(dsignpuf[0]+dsignpuf2)
    phinpf2 = phinpf[0]+du*(dphinpuf[0]+dphinpuf2)
    
    dphinpuf2 = 1/(rnpf2)*(rnpf2[0]*0.0-dv*trapint(drnpuf2*dphinpvf2,initial=0))
    Sigfuncnp = odeu.Sigfunc(drnpvf2,drnpuf2,dphinpuf2,dphinpvf2,rnpf2,signpf2,Q)
    dsignpuf2 = dsignpuf[1][0]+dv*trapint(Sigfuncnp,initial=0)
    ###
    
    ################
    ###Corrector###
    ################
    drnpvf[1] = drnpvf[0]+1/2*du*(odeu.Rfunc(drnpvf[0],drnpuf[0],rnpf[0],signpf[0],Q,Lambda)+odeu.Rfunc(drnpvf[1],drnpuf[1],rnpf[1],signpf[1],Q,Lambda))
    dsignpvf[1] = dsignpvf[0]+1/2*du*(odeu.Sigfunc(drnpvf[0],drnpuf[0],dphinpuf[0],dphinpvf[0],rnpf[0],signpf[0],Q)+odeu.Sigfunc(drnpvf[1],drnpuf[1],dphinpuf[1],dphinpvf[1],rnpf[1],signpf[1],Q))
    dphinpvf[1] = dphinpvf[0]+1/2*du*(odeu.Phifunc(drnpvf[0],drnpuf[0],dphinpuf[0],dphinpvf[0],rnpf[0])+odeu.Phifunc(drnpvf[1],drnpuf[1],dphinpuf[1],dphinpvf[1],rnpf[1]))
    drnpuf[1] = drnpuf[0]+1/2*du*(odeu.Coneq(drnpuf[0],dsignpuf[0],dphinpuf[0],rnpf[0])+odeu.Coneq(drnpuf[1],dsignpuf[1],dphinpuf[1],rnpf[1]))
    
    rnpf[1] = rnpf[0]+1/2*du*(drnpuf[0]+drnpuf[1])
    
    signpf[1] = signpf[0]+1/2*du*(dsignpuf[0]+dsignpuf[1])
    phinpf[1] = phinpf[0]+1/2*du*(dphinpuf[0]+dphinpuf[1])
    
    if ubdytype=='edd':
        #dsignpuf[1][0]=(np.log(abs(1-2*M0/rnpf[1][0]+Q**2.0/rnpf[1][0]-Lambda*rnpf[1][0]**2.0/3))-signpf[0][0])/du
        dsignpuf[1][0] = 2*(3*Q**2-3*M0*rnpf[1][0]+rnpf[1][0]**4*Lambda)/(rnpf[1][0]*(-3*Q**2+rnpf[1][0]*(6*M0-3*rnpf[1][0]+rnpf[1][0]**3*Lambda)))*drnpuf[1][0]
    
    dphinpuf[1] = 1/(rnpf[1])*(rnpf[1][0]*dphinpuf[1][0]-dv*trapint(drnpuf[1]*dphinpvf[1],initial=0))
    Sigfuncnp = odeu.Sigfunc(drnpvf[1],drnpuf[1],dphinpuf[1],dphinpvf[1],rnpf[1],signpf[1],Q)
    dsignpuf[1] = dsignpuf[1][0]+dv*trapint(Sigfuncnp,initial=0)
    
    massnpf[1] = (1+4.0*np.exp(-signpf[1])*drnpuf[1]*drnpvf[1])*rnpf[1]/2.0+np.power(Q,2.0)/(2.0*rnpf[1])-Lambda*np.power(rnpf[1],3.0)/6.0
    #####
    diff1=np.nanmax(np.abs((drnpuf[1]-drnpuf2)))#*4/3*(du)**(-1)))
    diff2=np.nanmax(np.abs((dsignpuf[1]-dsignpuf2)))#*4/3*(du)**(-1)))
    diff3=np.nanmax(np.abs((dphinpuf[1]-dphinpuf2)))#*4/3*(du)**(-1)))
    diff4=np.nanmax(np.abs((rnpf[1]-rnpf2)))#*4/3*(du)**(-1)))
    diff5=np.nanmax(np.abs((signpf[1]-signpf2)))#*4/3*(du)**(-1)))
    diff6=np.nanmax(np.abs((phinpf[1]-phinpf2)))#*4/3*(du)**(-1)))
    diff7=np.nanmax(np.abs((drnpvf[1]-drnpvf2)))#*4/3*(du)**(-1)))
    diff8=np.nanmax(np.abs((dsignpvf[1]-dsignpvf2)))#*4/3*(du)**(-1)))
    diff9=np.nanmax(np.abs((dphinpvf[1]-dphinpvf2)))#*4/3*(du)**(-1)))
    
    
    #print(diff1,diff2,diff3,diff4,diff5,diff6,diff7,diff8,diff9)
    diff=np.nanmax([diff1,diff2,diff3,diff4,diff5,diff6,diff7])
    #print(du,diff)
    ###
    
    

    if icount==uscal and diff< Tol:
        urange=np.append(urange,urange[i]+du)
        rnp.append(rnpf[1].reshape(1,Nv))
        signp.append(signpf[1].reshape(1,Nv))
        phinp.append(phinpf[1].reshape(1,Nv))
        massnp.append(massnpf[1].reshape(1,Nv))
        if extraterms==True:
            drnpu.append(drnpuf[1].reshape(1,Nv))
            drnpv.append(drnpvf[1].reshape(1,Nv))
            dsignpu.append(dsignpuf[1].reshape(1,Nv))
            dsignpv.append(dsignpvf[1].reshape(1,Nv))
            dphinpu.append(dphinpuf[1].reshape(1,Nv))
            dphinpv.append(dphinpvf[1].reshape(1,Nv))
        icount=0

        print("%.24f" % urange[i])
        #print("%.24f" % du)
        print(du)
        print(drnpuf[1][-1],dsignpuf[1][-1],dphinpuf[1][-1])
        #print(dsignpuf[1][-1])
        print(drnpvf[1][-1],dsignpvf[1][-1],dphinpvf[1][-1])
        print(odeu.Rfunc(drnpvf[1][-1],drnpuf[1][-1],rnpf[1][-1],signpf[1][-1],Q,Lambda),odeu.Coneq(drnpuf[1][-1],dsignpuf[1][-1],dphinpuf[1][-1],rnpf[1][-1]),odeu.Sigfunc(drnpvf[1][-1],drnpuf[1][-1],dphinpuf[1][-1],dphinpvf[1][-1],rnpf[1][-1],signpf[1][-1],Q))
        print('-')
        print(diff1,diff2,diff3)
        print(diff4,diff5,diff6)
        print(diff7,diff8,diff9)
        print(massnpf[1][-2])
        print('---')
        i+=1
        du=du*2
    elif diff>Tol:
        du=du/2
        #print(du)
        #print(diff)
    else:
        urange[i]=urange[i]+du
        #du=du*2
        icount+=1
    
    if diff<Tol:
        if ubdytype=='adapt':
            maxsig3=maxsig2
            maxsig2=maxsig1
            maxsig1=signpf[1][0]-np.nanmax(signpf[1])


        #break
        rnpf[0]=rnpf[1]
        phinpf[0]=phinpf[1]
        signpf[0]=signpf[1]
        drnpvf[0]=drnpvf[1]
        dsignpvf[0]=dsignpvf[1]
        dphinpvf[0]=dphinpvf[1]
        drnpuf[0]=drnpuf[1]
        dsignpuf[0]=dsignpuf[1]
        dphinpuf[0]=dphinpuf[1]

In [ ]:
rnpfile.close()
signpfile.close()
phinpfile.close()
massnpfile.close()
if extraterms==True:
    drnpufile.close()
    drnpvfile.close()
    dsignpufile.close()
    dsignpvfile.close()
    dphinpufile.close()
    dphinpvfile.close()
    
Nu=i+1

In [ ]:
print(len(rnpf[0,:]))
print(len(rnpf))
print(Nu)
print(len(urange))
plt.plot(vrange,drnpvf[0],'--')
plt.plot(vrange,np.abs(drnpvf[0]))
plt.plot([vrange[0],vrange[-1]],[.5,.5],'g--',linewidth=.5)
#plt.plot(vrange,np.abs(drnpvf[1]))
plt.yscale('log')
plt.grid()
plt.show()
print(rnpf[0,0])


In [ ]:
plt.plot(urange)
plt.show()

In [ ]:
switch=0.0
np.save('rnputil',np.array([Nu*Nv,Nu,Nv,ru0,dr0v,M0,Q,Lambda,wHD,switch,vbdytype]))
np.save('urange',urange)
np.save('vrange',vrange)

In [ ]:
stop = timeit.default_timer()

In [ ]:
acttime=stop - start

numpoints=Nu*Nv*uscal

print(str(acttime)+' seconds')
print(str((acttime)/60)+' minutes')
print(str((acttime)/3600)+' hours')

print(str(numpoints)+' points')

print(str((numpoints)/(stop - start))+' points per second') 

conv=False
j=0
tempanswer=[]
    while conv==False:
        dv=TempTolv*dv0
        jcount=1
        while j<Nv-1:
            j1=mth.ceil(jcount*TempTolv)
            j2=mth.floor(jcount*TempTolv)
            answer=dnu.x4giveralt(0,j+mth.ceil((jcount-1)*TempTolv),du,dv*TempTolv,rnpf,phinpf,signpf,massnpf,M0,Q,Lambda,datatype)
            rnpf[1][j+j1]=answer[0]
            phinpf[1][j+j1]=answer[1]
            signpf[1][j+j1]=answer[2]
            massnpf[1][j+j1]=answer[3]
            #drunptemp[j+1]=answer[4]
            dsigunpv=answer[5]
            jcount=(jcount+1)**(0**j2)
            j=j+j2
        
        if np.abs(tempanswer[0]-answer[0])/answer[0]>0.01 or not tempanswer:
            tempanswer=answer
            TempTol=TempTolv/2
            
        else:
            conv=True
            